In [8]:
import importlib
import math
import sklearn, sklearn.linear_model, torch
import pandas as pd, numpy as np

lat = pd.read_csv("/kaggle/input/gd2022datasets/bls-emp_by_cfips.csv")
display(lat)
lat['row_id'] = lat.apply(lambda x:f"{x['cfips']}_{x['first_day_of_month']}", axis = 1)
la = lat.set_index("row_id")

basedir = "/kaggle/input/godaddy-microbusiness-density-forecasting/"

daycols = ['2019-08-01', '2019-09-01',
           '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01',
           '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01',
           '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
           '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01',
           '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01',
           '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01',
           '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
           '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01']
class gd:
    def get_base(pdtime = True):
        d = pd.read_csv(f"{basedir}/train.csv")
        d2 = pd.read_csv(f"{basedir}/revealed_test.csv")
        d = pd.concat([d,d2])
        if pdtime:
            d['first_day_of_month'] = pd.to_datetime(d['first_day_of_month'])
        return d
    
bdata = pd.concat([gd.get_base()]).set_index("row_id")
bdata = bdata.sort_values(['cfips', 'first_day_of_month'])
bdata = bdata.query("cfips != 15005 and cfips != 28055 and cfips != 48301") #15005 not in bls, 28055 "Issaquena County" and 48301 is loving county.  These are 0 active counties and not helpful
#, is small active going from 1 to 0, 0 to 1 so pct is NaN
#these have already been removed, here for doc
#notin_bls = ['15005_2019-08-01', '15005_2019-09-01', '15005_2019-10-01', '15005_2019-11-01', '15005_2019-12-01', '15005_2020-01-01', '15005_2020-02-01', '15005_2020-03-01', '15005_2020-04-01', '15005_2020-05-01', '15005_2020-06-01', '15005_2020-07-01', '15005_2020-08-01', '15005_2020-09-01', '15005_2020-10-01', '15005_2020-11-01', '15005_2020-12-01', '15005_2021-01-01', '15005_2021-02-01', '15005_2021-03-01', '15005_2021-04-01', '15005_2021-05-01', '15005_2021-06-01', '15005_2021-07-01', '15005_2021-08-01', '15005_2021-09-01', '15005_2021-10-01', '15005_2021-11-01', '15005_2021-12-01', '15005_2022-01-01', '15005_2022-02-01', '15005_2022-03-01', '15005_2022-04-01', '15005_2022-05-01', '15005_2022-06-01', '15005_2022-07-01', '15005_2022-08-01', '15005_2022-09-01', '15005_2022-10-01']

holdout_split_date = '2023-05-01'
bdata_holdout = bdata.query(f"first_day_of_month >= '{holdout_split_date}'")
bdata = bdata.query(f"first_day_of_month < '{holdout_split_date}'") #remove month for validation

bdata['month'] = bdata['first_day_of_month'].apply(lambda x:x.month - 1)
bdata["x_norm"] = 2 * math.pi * bdata["month"] / bdata["month"].max()
bdata["cos_month"] = np.cos(bdata["x_norm"])
bdata["sin_month"] = np.cos(bdata["x_norm"])
bdataind_for_bls = bdata.index
bdata['unemployment_rate'] = la.query("measure_type == 'Unemployment rate'").loc[bdataind_for_bls, 'value']   
bdata['unemployment'] = la.query("measure_type == 'Unemployment'").loc[bdataind_for_bls, 'value'] 
bdata['employment'] = la.query("measure_type == 'Employment'").loc[bdataind_for_bls, 'value']  
bdata['labor_force'] = la.query("measure_type == 'Labor force'").loc[bdataind_for_bls, 'value']
tday = '5039_2021-02-01'
display(bdata.loc[tday, bdata.columns[-4:]])
display(la.loc[tday])
for i in range(12):
    p = i+1
    bdata[f'l{p}_active'] = bdata.groupby("cfips")['active'].pct_change(periods = p) + 1
    bdata[f'l{p}_abs_active'] = np.abs(bdata.groupby("cfips")['active'].pct_change(periods = p))
    bdata[f'l{p}_ur'] = bdata.groupby("cfips")['unemployment_rate'].pct_change(periods = p) + 1
    bdata[f'l{p}_uemp'] = bdata.groupby("cfips")['unemployment'].pct_change(periods = p) + 1
    bdata[f'l{p}_emp'] = bdata.groupby("cfips")['employment'].pct_change(periods = p) + 1
    bdata[f'l{p}_lf'] = bdata.groupby("cfips")['labor_force'].pct_change(periods = p) + 1

bdata['pop'] = np.round(bdata['active']/(bdata['microbusiness_density']/100))
total_pop = bdata['active'].sum()
bdata['pop_share'] = len(bdata)*bdata['active']/total_pop

for r in range(0,5):
    bdata = bdata.copy()
    i = r + 1
    bdata[f'shiftm{i}_l{i}_active'] =  bdata.groupby("cfips")[f'l{i}_active'].shift(-i)
    bdata[f'shiftm{i}_l{i}_abs_active'] =  bdata.groupby("cfips")[f'l{i}_abs_active'].shift(-i)
    bdata[f'shiftm{i}_l{i}_active_logit'] =  bdata[f'shiftm{i}_l{i}_active'] != 1
    bdata[f'shiftm{i}_l{i}_active_logitx'] =  bdata[f'shiftm{i}_l{i}_abs_active'] > 0.007
    bdata[f'shiftm{i}_l{i}_active_logitx_out'] =  bdata[f'shiftm{i}_l{i}_abs_active'] > 0.015
    bdata[f'shiftm{i}_l{i}_active_logitx_out_out'] =  bdata[f'shiftm{i}_l{i}_abs_active'] > 0.05
    bdata[f'shiftf{i}_l1_active'] =  bdata.groupby("cfips")['l1_active'].shift(i)
    bdata[f'shiftf{i}_l1_abs_active'] =  bdata.groupby("cfips")['l1_abs_active'].shift(i)
    bdata[f'shiftf{i}_l1_active_logit'] =  bdata[f'shiftf{i}_l1_active'] != 1
    bdata[f'shiftf{i}_l1_active_logitx'] =  bdata[f'shiftf{i}_l1_abs_active'] > 0.007
    bdata[f'shiftm{i}_pop'] =  bdata.groupby("cfips")['pop'].shift(-i)
    bdata[f'shiftm{i}_mbd'] =  bdata.groupby("cfips")['microbusiness_density'].shift(-i)
    bdata[f'shiftm{i}_active'] =  bdata.groupby("cfips")['active'].shift(-i)

bdata['voly'] = (bdata['shiftf1_l1_active']-1)*0.5 + 0.9+(bdata['l1_active']-1)

#bdata = bdata.query("first_day_of_month >= '2020-08-01'")
census = pd.read_csv("/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv").set_index("cfips")
bdata = bdata.join(census, how='left', on='cfips')

bdata['year'] = bdata['first_day_of_month'].apply(lambda x:x.year)
bdata.to_pickle("base_data_with_employment_and_pcts_valid_zero_months.pkl")
bdata_holdout.to_pickle("base_data_valid_zero_months.pkl")
print(list(bdata.columns))
from IPython.display import HTML
display(HTML(bdata.iloc[-10:].to_html(max_cols=500)))
print("max date", bdata.first_day_of_month.max(),"min_date", bdata.first_day_of_month.min())


,series_id,year,period,value,footnote_codes,cfips,first_day_of_month,measure_type
0,LAUCN010010000000003,2019,M01,3.9,NaN,1001,2019-01-01,Unemployment rate
1,LAUCN010010000000003,2019,M02,3.4,NaN,1001,2019-02-01,Unemployment rate
2,LAUCN010010000000003,2019,M03,3.3,NaN,1001,2019-03-01,Unemployment rate
3,LAUCN010010000000003,2019,M04,2.5,NaN,1001,2019-04-01,Unemployment rate
4,LAUCN010010000000003,2019,M05,2.4,NaN,1001,2019-05-01,Unemployment rate
...,...,...,...,...,...,...,...,...
656823,LAUCN721530000000006,2022,M08,10276.0,P,72153,2022-08-01,Labor force
656824,LAUCN721530000000006,2022,M09,NaN,N,72153,2022-09-01,Labor force
656825,LAUCN721530000000006,2022,M10,10539.0,NaN,72153,2022-10-01,Labor force
656826,LAUCN721530000000006,2022,M11,10802.0,NaN,72153,2022-11-01,Labor force


unemployment_rate       4.9
unemployment          136.0
employment           2649.0
labor_force          2785.0
Name: 5039_2021-02-01, dtype: object

,series_id,year,period,value,footnote_codes,cfips,first_day_of_month,measure_type
row_id,,,,,,,,
5039_2021-02-01,LAUCN050390000000003,2021,M02,4.9,NaN,5039,2021-02-01,Unemployment rate
5039_2021-02-01,LAUCN050390000000004,2021,M02,136.0,NaN,5039,2021-02-01,Unemployment
5039_2021-02-01,LAUCN050390000000005,2021,M02,2649.0,NaN,5039,2021-02-01,Employment
5039_2021-02-01,LAUCN050390000000006,2021,M02,2785.0,NaN,5039,2021-02-01,Labor force


['cfips', 'county', 'state', 'first_day_of_month', 'microbusiness_density', 'active', 'month', 'x_norm', 'cos_month', 'sin_month', 'unemployment_rate', 'unemployment', 'employment', 'labor_force', 'l1_active', 'l1_abs_active', 'l1_ur', 'l1_uemp', 'l1_emp', 'l1_lf', 'l2_active', 'l2_abs_active', 'l2_ur', 'l2_uemp', 'l2_emp', 'l2_lf', 'l3_active', 'l3_abs_active', 'l3_ur', 'l3_uemp', 'l3_emp', 'l3_lf', 'l4_active', 'l4_abs_active', 'l4_ur', 'l4_uemp', 'l4_emp', 'l4_lf', 'l5_active', 'l5_abs_active', 'l5_ur', 'l5_uemp', 'l5_emp', 'l5_lf', 'l6_active', 'l6_abs_active', 'l6_ur', 'l6_uemp', 'l6_emp', 'l6_lf', 'l7_active', 'l7_abs_active', 'l7_ur', 'l7_uemp', 'l7_emp', 'l7_lf', 'l8_active', 'l8_abs_active', 'l8_ur', 'l8_uemp', 'l8_emp', 'l8_lf', 'l9_active', 'l9_abs_active', 'l9_ur', 'l9_uemp', 'l9_emp', 'l9_lf', 'l10_active', 'l10_abs_active', 'l10_ur', 'l10_uemp', 'l10_emp', 'l10_lf', 'l11_active', 'l11_abs_active', 'l11_ur', 'l11_uemp', 'l11_emp', 'l11_lf', 'l12_active', 'l12_abs_active', 

,cfips,county,state,first_day_of_month,microbusiness_density,active,month,x_norm,cos_month,sin_month,unemployment_rate,unemployment,employment,labor_force,l1_active,l1_abs_active,l1_ur,l1_uemp,l1_emp,l1_lf,l2_active,l2_abs_active,l2_ur,l2_uemp,l2_emp,l2_lf,l3_active,l3_abs_active,l3_ur,l3_uemp,l3_emp,l3_lf,l4_active,l4_abs_active,l4_ur,l4_uemp,l4_emp,l4_lf,l5_active,l5_abs_active,l5_ur,l5_uemp,l5_emp,l5_lf,l6_active,l6_abs_active,l6_ur,l6_uemp,l6_emp,l6_lf,l7_active,l7_abs_active,l7_ur,l7_uemp,l7_emp,l7_lf,l8_active,l8_abs_active,l8_ur,l8_uemp,l8_emp,l8_lf,l9_active,l9_abs_active,l9_ur,l9_uemp,l9_emp,l9_lf,l10_active,l10_abs_active,l10_ur,l10_uemp,l10_emp,l10_lf,l11_active,l11_abs_active,l11_ur,l11_uemp,l11_emp,l11_lf,l12_active,l12_abs_active,l12_ur,l12_uemp,l12_emp,l12_lf,pop,pop_share,shiftm1_l1_active,shiftm1_l1_abs_active,shiftm1_l1_active_logit,shiftm1_l1_active_logitx,shiftm1_l1_active_logitx_out,shiftm1_l1_active_logitx_out_out,shiftf1_l1_active,shiftf1_l1_abs_active,shiftf1_l1_active_logit,shiftf1_l1_active_logitx,shiftm1_pop,shiftm1_mbd,shiftm1_active,shiftm2_l2_active,shiftm2_l2_abs_active,shiftm2_l2_active_logit,shiftm2_l2_active_logitx,shiftm2_l2_active_logitx_out,shiftm2_l2_active_logitx_out_out,shiftf2_l1_active,shiftf2_l1_abs_active,shiftf2_l1_active_logit,shiftf2_l1_active_logitx,shiftm2_pop,shiftm2_mbd,shiftm2_active,shiftm3_l3_active,shiftm3_l3_abs_active,shiftm3_l3_active_logit,shiftm3_l3_active_logitx,shiftm3_l3_active_logitx_out,shiftm3_l3_active_logitx_out_out,shiftf3_l1_active,shiftf3_l1_abs_active,shiftf3_l1_active_logit,shiftf3_l1_active_logitx,shiftm3_pop,shiftm3_mbd,shiftm3_active,shiftm4_l4_active,shiftm4_l4_abs_active,shiftm4_l4_active_logit,shiftm4_l4_active_logitx,shiftm4_l4_active_logitx_out,shiftm4_l4_active_logitx_out_out,shiftf4_l1_active,shiftf4_l1_abs_active,shiftf4_l1_active_logit,shiftf4_l1_active_logitx,shiftm4_pop,shiftm4_mbd,shiftm4_active,shiftm5_l5_active,shiftm5_l5_abs_active,shiftm5_l5_active_logit,shiftm5_l5_active_logitx,shiftm5_l5_active_logitx_out,shiftm5_l5_active_logitx_out_out,shiftf5_l1_active,shiftf5_l1_abs_active,shiftf5_l1_active_logit,shiftf5_l1_active_logitx,shiftm5_pop,shiftm5_mbd,shiftm5_active,voly,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2017,pct_foreign_born_2018,pct_foreign_born_2019,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,year
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
56045_2022-03-01,56045,Weston County,Wyoming,2022-03-01,1.767542,99,2,1.142397,0.415415,0.415415,3.1,116.0,3606.0,3722.0,1.010204,0.010204,1.000000,1.017544,0.995857,0.996519,1.010204,0.010204,1.068966,1.064220,0.993115,0.995187,0.990000,0.010000,1.291667,1.318182,0.986864,0.994655,1.010204,0.010204,1.240000,1.234043,0.975385,0.981799,1.064516,0.064516,1.240000,1.260870,0.985785,0.992533,1.053191,0.053191,1.240000,1.260870,0.995308,1.001884,1.064516,0.064516,1.107143,1.104762,0.992568,0.995720,1.076087,0.076087,1.000000,1.035714,1.021819,1.022247,1.076087,0.076087,0.885714,0.878788,0.996133,0.992004,1.053191,0.053191,0.861111,0.852941,0.996683,0.991476,1.031250,0.031250,0.861111,0.872180,1.001110,0.996519,1.010204,0.010204,0.794872,0.800000,0.998892,0.991212,5601.0,0.015308,1.000000,0.000000,False,False,False,False,1.000000,0.000000,False,False,5601.0,1.767542,99.0,1.020202,0.020202,True,True,True,False,0.980000,0.020000,True,True,5601.0,1.803249,101.0,1.020202,0.020202,True,True,True,False,1.020408,0.020408,True,True,5601.0,1.803249,101.0,1.020202,0.020202,True,True,True,False,1.053763,0.053763,True,True,5601.0,1.803249,101.0,1.010101,0.010

max date 2022-12-01 00:00:00 min_date 2019-08-01 00:00:00


In [13]:

#3132 because of 3 problematic counties with either 0 active or missing in the bls dataset 
model = torch.nn.Linear(3132, 3132) 
#I did a lot of different submits to the LB using nn, this has mappings from dec -> january LB overfit
#More information here - https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/discussion/413109
#The largest signal informing probing was volatile counties with greatest previous month change, see code below for more
model.load_state_dict(torch.load('/kaggle/input/gd23subs/model_weights.pth'))

#This has various lag features based on the training plus
#what I publicly shared at the outset of the comp - https://www.kaggle.com/datasets/kaggleqrdl/gd2022datasets
#see https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/discussion/395131 for an explanation
bdata = pd.read_pickle("/kaggle/input/gd23subs/base_data_with_employment_and_pcts_valid_zero_months.pkl") 

#Population data from census website
pops = pd.read_pickle("/kaggle/input/gd23subs/pops.pkl")

def active_to_md(valc):
    valc['2023pops'] = 0
    for i,r in valc.iterrows():
        valc.loc[i,'2023pops'] = pops.loc[(i,2023)]
    valc['microbusiness_density_pr'] = 100*valc['preds_active_rnd']/valc['2023pops']

#hat tip cdeotte
def smape(y_true, y_pred):
    # CONVERT TO NUMPY
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # WHEN BOTH EQUAL ZERO, METRIC IS ZERO
    both = np.abs(y_true) + np.abs(y_pred)
    idx = np.where(both==0)[0]
    y_true[idx]=1; y_pred[idx]=1
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

#This is just a display func to validate that values are same as subs
def cmp_valc(val, dt, fname="/kaggle/input/sub2cnd/final_cand1 (1).csv"):
    #fname default is the sub which placed #1
    cmp = pd.read_csv(fname)
    cmp['first_day_of_month'] = cmp.apply(lambda x:x['row_id'][-10:], axis = 1)
    cmp = cmp.set_index("row_id")
    valc = val.copy().reset_index()
    valc['row_id'] = valc.apply(lambda x:f"{x.cfips}_{dt}", axis=1)
    valc=valc.set_index("row_id")
    ind = valc.index
    print(f"smape against winning sub for {dt} len:{len(ind)}:", smape(cmp.loc[ind]['microbusiness_density'], valc['microbusiness_density_pr']))

    
val = bdata.query("first_day_of_month == '2022-12-01'").copy().set_index("cfips")
#january overfit active values
X = np.array(val['active'])
val['active'] = model(torch.Tensor(X).T) .detach().numpy()
val['preds_active_rnd'] = val['active']
active_to_md(val)
#There's a slight error here against my public lb #1 sub, 
#I think it's because I submitted some more to stay
#at number one on the LB during the 3 month wait, but didn't update my baseline  
cmp_valc(val, "2023-01-01", fname="/kaggle/input/datasub/final_3m_cand (1).csv")

val = val.reset_index()
val = val.sort_values("cfips", ascending = False)


def get_skip(fcs, tc, dt, cmpdt):
    global val
    model = sklearn.linear_model.LinearRegression()
    #I used 2020-08 as the beginning because of NaN values due to lag calculations.
    #Thinking about it now, I probably could have used dropna here instead
    tr = bdata.query(f"first_day_of_month <= '{dt}' and first_day_of_month >= '2020-08-01'").copy()
    model.fit(tr[fcs], tr[tc])
    
    valc = bdata.query("first_day_of_month == '2022-11-01'").copy()
    #This line is very important.  We are projecting against the overfit above, and rounding which is critical.
    valc['preds_active_rnd'] = np.round(valc['active'] * model.predict(valc[fcs]))
    valc = valc.set_index("cfips")
    active_to_md(valc)
    cmp_valc(valc, cmpdt)
    return valc

#these are just linear regression models with these features.
#as per below, no error against sub.  See https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/discussion/395131
#for an explanation on how the features were chosen.  
skip4may = get_skip(['pct_college_2021', 'l11_uemp', 'shiftf1_l1_active'], 'shiftm4_l4_active', '2022-08-01', "2023-05-01")
skip3apr = get_skip(['shiftf1_l1_active_logit', 'pct_college_2019', 'l4_active', 'l10_lf'], 'shiftm3_l3_active', '2022-09-01',"2023-04-01")
skip2mar = get_skip(['shiftf5_l1_active_logit', 'pct_college_2020', 'shiftf3_l1_active_logit',
                    'l6_emp', 'shiftf4_l1_active_logit','shiftf2_l1_active'], 'shiftm2_l2_active', '2022-09-01',"2023-03-01")

#In general, the data was mostly non stationary except for some global growth term, so basic linear models were good. 

smape against winning sub for 2023-01-01 len:3132: 0.014762481704571312
smape against winning sub for 2023-05-01 len:3132: 1.9522765966459463
smape against winning sub for 2023-04-01 len:3132: 1.929143239178773
smape against winning sub for 2023-03-01 len:3132: 1.9006436238972642


In [14]:
import sklearn, sklearn.linear_model, torch
import pandas as pd, numpy as np

#3132 because of 3 problematic counties with either 0 active or missing in the bls dataset 
model = torch.nn.Linear(3132, 3132) 
#I did a lot of different submits to the LB using nn, this has mappings from dec -> january LB overfit
#More information here - https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/discussion/413109
#The largest signal informing probing was volatile counties with greatest previous month change, see code below for more
model.load_state_dict(torch.load('/kaggle/input/gd23subs/model_weights.pth'))

#This has various lag features based on the training plus
#what I publicly shared at the outset of the comp - https://www.kaggle.com/datasets/kaggleqrdl/gd2022datasets
#see https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/discussion/395131 for an explanation
bdata = pd.read_pickle("/kaggle/input/gd23subs/base_data_with_employment_and_pcts_valid_zero_months.pkl") 

#Population data from census website
pops = pd.read_pickle("/kaggle/input/gd23subs/pops.pkl")

def active_to_md(valc):
    valc['2023pops'] = 0
    for i,r in valc.iterrows():
        valc.loc[i,'2023pops'] = pops.loc[(i,2023)]
    valc['microbusiness_density_pr'] = 100*valc['preds_active_rnd']/valc['2023pops']

#hat tip cdeotte
def smape(y_true, y_pred):
    # CONVERT TO NUMPY
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # WHEN BOTH EQUAL ZERO, METRIC IS ZERO
    both = np.abs(y_true) + np.abs(y_pred)
    idx = np.where(both==0)[0]
    y_true[idx]=1; y_pred[idx]=1
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

#This is just a display func to validate that values are same as subs
def cmp_valc(val, dt, fname="/kaggle/input/sub2cnd/final_cand1 (1).csv"):
    #fname default is the sub which placed #1
    cmp = pd.read_csv(fname)
    cmp['first_day_of_month'] = cmp.apply(lambda x:x['row_id'][-10:], axis = 1)
    cmp = cmp.set_index("row_id")
    valc = val.copy().reset_index()
    valc['row_id'] = valc.apply(lambda x:f"{x.cfips}_{dt}", axis=1)
    valc=valc.set_index("row_id")
    ind = valc.index
    print(f"smape against winning sub for {dt} len:{len(ind)}:", smape(cmp.loc[ind]['microbusiness_density'], valc['microbusiness_density_pr']))

    
val = bdata.query("first_day_of_month == '2022-12-01'").copy().set_index("cfips")
#january overfit active values
X = np.array(val['active'])
val['active'] = model(torch.Tensor(X).T) .detach().numpy()
val['preds_active_rnd'] = val['active']
active_to_md(val)
#There's a slight error here against my public lb #1 sub, 
#I think it's because I submitted some more to stay
#at number one on the LB during the 3 month wait, but didn't update my baseline  
cmp_valc(val, "2023-01-01", fname="/kaggle/input/datasub/final_3m_cand (1).csv")

val = val.reset_index()
val = val.sort_values("cfips", ascending = False)


def get_skip(fcs, tc, dt, cmpdt):
    global val
    model = sklearn.linear_model.LinearRegression()
    #I used 2020-08 as the beginning because of NaN values due to lag calculations.
    #Thinking about it now, I probably could have used dropna here instead
    tr = bdata.query(f"first_day_of_month <= '{dt}' and first_day_of_month >= '2020-08-01'").copy()
    model.fit(tr[fcs], tr[tc])
    
    valc = val.copy()
    #This line is very important.  We are projecting against the overfit above, and rounding which is critical.
    valc['preds_active_rnd'] = np.round(valc['active'] * model.predict(valc[fcs]))
    valc = valc.set_index("cfips")
    active_to_md(valc)
    cmp_valc(valc, cmpdt)
    return valc

#these are just linear regression models with these features.
#as per below, no error against sub.  See https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/discussion/395131
#for an explanation on how the features were chosen.  
skip4may = get_skip(['pct_college_2021', 'l11_uemp', 'shiftf1_l1_active'], 'shiftm4_l4_active', '2022-08-01', "2023-05-01")
skip3apr = get_skip(['shiftf1_l1_active_logit', 'pct_college_2019', 'l4_active', 'l10_lf'], 'shiftm3_l3_active', '2022-09-01',"2023-04-01")
skip2mar = get_skip(['shiftf5_l1_active_logit', 'pct_college_2020', 'shiftf3_l1_active_logit',
                    'l6_emp', 'shiftf4_l1_active_logit','shiftf2_l1_active'], 'shiftm2_l2_active', '2022-09-01',"2023-03-01")

#In general, the data was mostly non stationary except for some global growth term, so basic linear models were good. 

smape against winning sub for 2023-01-01 len:3132: 0.014762481704571312
smape against winning sub for 2023-05-01 len:3132: 2.3459492217422063e-15
smape against winning sub for 2023-04-01 len:3132: 2.497315461459175e-15
smape against winning sub for 2023-03-01 len:3132: 2.376090999994613e-15


In [12]:
valc = skip3apr.copy()
vf = pd.read_csv("/kaggle/input/md-march-23-go-daddy/VF_md_counties_Q123.csv").set_index("cfips")
valc = valc[~valc.index.isin([2195, 2275])]
print("dummy nov->mar", smape(vf.loc[valc.index].active_mar23, valc['preds_active_rnd']))

dummy nov->mar 3.8194918352579754


In [1]:
import numpy as np
def dif(newv, base, prev,guess, k=100/3135):
    p1 = prev
    p2 = guess
    c = base - newv
    #c is p1 score - p2 score, p1 score being lower/better
    v1 = c*c*p1*p1-2*c*c*p1*p2+c*c*p2*p2+64*k*k*p1*p2
    v1 = np.sqrt(v1)
    r1 = (-v1 - c*p1-c*p2)/(2*(c-4*k))
    r2 = (-v1 - c*p1-c*p2)/(2*(c+4*k))
    r3 = (v1 - c*p1-c*p2)/(2*(c-4*k))
    r4 = (v1 - c*p1-c*p2)/(2*(c+4*k))
    
    v = ((c*p1+c*p2+4*k*p1-4*k*p2)**2 - 4*c*c*p1*p2)
    v = np.sqrt(v)
    r5 = (-v - c*p1-c*p2-4*k*p1+4*k*p2)/(2*c)
    r6 = (v - c*p1-c*p2-4*k*p1+4*k*p2)/(2*c)
    
    v = ((c*p1+c*p2-4*k*p1+4*k*p2)**2 - 4*c*c*p1*p2)
    v = np.sqrt(v)
    r7 = (-v - c*p1-c*p2+4*k*p1-4*k*p2)/(2*c)
    r8 = (v - c*p1-c*p2+4*k*p1-4*k*p2)/(2*c)
    rts = [r1, r2, r3,r4,r5,r6,r7,r8]
    roots = str(rts)+"\n"    
    if prev < guess:
        return roots+f"prev:{prev} < guess:{guess}, so prev<grnd & guess>grnd:{r1}, prev<grnd & guess<grnd:{r6}, prev>grnd & guess>grnd:{r8}", (r1,r6,r8)
    else:
        return roots+f"prev:{prev} > guess:{guess}, so prev>grnd & guess<grnd:{r4}, prev<grnd & guess<grnd:{r5}, prev>grnd & guess>grnd:{r7}", (r4,r5,r7)
    #r1 if p1 < p2 and p1 < g, p2 > g
    #r4 if p1 > p2 and p1 > g, p2 < g
    #r5 if p1 > p2 and p1 < g, p2 < g
    #r6 if p1 < p2 and p1 < g, p2 < g
    #r7 if p1 > p2 and p1 > g, p2 > g
    #r8 if p1 < p2 and p1 > g, p2 > g

    return r1, r2, r3,r4,r5,r6,r7,r8

def pcta(v1, v2):
    return np.abs(v1-v2)/v2

cfiplh = []
def low_high(cfip, prev, sch, gh, scl, gl, base = 1.4631, out = False, setdf = None):
    cfiplh.append(cfip)
    if setdf is not None:
        _ = [print("{:0.5f}".format(setdf.loc[f'{cfip}_2022-{m}-01','microbusiness_density']), end ="|") for m in ["12","11","10","09","08","07"]]
        print("")
    if out:
        print(f"low high cfip:{cfip} prevval:{prev} sch:{sch} gh:{gh} scl:{scl} gl:{gl} base:{base} b-h:{base-sch} b-l:{base-scl}")
    mh,rtsh = dif(sch ,base,prev, gh)
    ml,rtsl = dif(scl ,base,prev, gl)
    if out:
        print(cfip, mh)
        print(cfip, ml)
    md = 99
    mdh = 99
    mdl = 99
    df,dfh,dfl = 100,100,100
    for i in range(3):
        for j in range(3):
            if np.isnan(rtsh[i]) or np.isnan(rtsl[j]):
                continue
            if rtsh[i] < 0 or rtsl[j] < 0:
                continue
            if i != j:
                if not np.isnan(rtsh[i]) and not np.isnan(rtsh[j]) and rtsh[j] > 0 and rtsh[j] > 0:                    
                    dfh = pcta(rtsh[i],rtsh[j])
                    if dfh < mdh:
                        besthi = i
                        besthj = j
                        mdh = dfh
                if not np.isnan(rtsl[i]) and not np.isnan(rtsl[j]) and rtsl[j] > 0 and rtsl[j] > 0:                    
                    dfl = pcta(rtsl[i],rtsl[j])
                    if dfl < mdl:
                        bestli = i
                        bestlj = j
                        mdh = dfl
            df =  pcta(rtsh[i],rtsl[j]) 
            if out:
                print(df, rtsh[i], rtsl[j], md)
            if df < md:
                md = df
                besti = i
                bestj = j
    if mdl < 0.01 or mdh < 0.01:
        if pcta(rsth[besthi], rtsh[besti]) > 0.01:
            pass
        if pcta(rstl[besthj], rtsl[bestj]) > 0.01:
            pass
    best = (rtsh[besti] + rtsl[bestj])/2
    imprv =  (smape([best],[prev])*1/3135)
    if setdf is not None:
        setdf.loc[f'{cfip}_2023-01-01','microbusiness_density'] = best
    if out:
        print(f"use {best} for -{imprv} improvement to {base - imprv} sc as best is {rtsh[besti]} and {rtsl[bestj]}, md:{md} mdh:{mdh} mdl:{mdl}\n\n")
    return imprv

#The code above can be used to probe the LB.  By probing with 10x and 0.1x prev values, you can use formula above to get roots for SMAPE to get close approximations 
#to the exact value.
#I used the following formula to rank most likely to change significantly.  I did other studies, using LGBM/Linear Regression as well, but this worked out well.
#diff{st+1} and diff{st+2} is the change in md over the last and second to last month respectively
#trc['diff'] = (0.9*trc[f'diff{st+1}']) + (0.4*trc[f'diff{st+2}']) 
#In general, it was just the top 10 that had the most changes, and while I probed about 20 others or so, they didn't move the needle that much.  